In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install torch

from transformers import BertTokenizer, BertForSequenceClassification
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/EDA/Ai Prompt Generator/Ai Writting/fine_tuned_bert_model")
tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/EDA/Ai Prompt Generator/Ai Writting/fine_tuned_bert_model")

In [ ]:
import pandas as pd
import torch


# Select the column 'question' from the dataset
questions = data['question'].tolist()  # Convert the questions to a list

# Tokenize the questions
encoded_inputs = tokenizer(questions, padding=True, truncation=True, return_tensors="pt", max_length=512)

# Prepare the input tensors
input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]

# Set the device to use (e.g., CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the tensors to the specified device
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Predict the sentiments for the questions
with torch.no_grad():
    logits = model(input_ids, attention_mask=attention_mask)[0]
    predictions = torch.argmax(logits, dim=1).cpu().numpy()

# Add the predicted sentiments as a new column in the dataset
data['sentiment'] = predictions

# Save the updated dataset with the new sentiment column
data.to_csv("data_with_sentiments.csv", index=False)


In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

# Load the dataset into a pandas DataFrame
df = pd.read_csv("/content/drive/MyDrive/EDA/Ai Prompt Generator/Ai Writting/Conversation/topical_chat.csv")  # Replace with your dataset path
X = df['message']
y = df['sentiment']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize and encode the text data
def tokenize_text(text):
    input_ids = []
    attention_masks = []

    for sent in text:
        encoded_dict = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

# Tokenize the training and validation text data
train_inputs, train_masks = tokenize_text(X_train)
val_inputs, val_masks = tokenize_text(X_val)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the sentiment labels
train_labels = torch.tensor(label_encoder.fit_transform(y_train))
val_labels = torch.tensor(label_encoder.transform(y_val))

In [ ]:
# Decoding the labels
train_labels_decoded = label_encoder.inverse_transform(train_labels)
val_labels_decoded = label_encoder.inverse_transform(val_labels)

In [ ]:
# Get the unique labels
unique_labels = label_encoder.classes_

# Print the label IDs and their original names
for i, label in enumerate(unique_labels):
    print(f"Label ID: {i} => Original Name: {label}")


Label ID: 0 => Original Name:  Angry
Label ID: 1 => Original Name:  Curious to dive deeper
Label ID: 2 => Original Name:  Disgusted
Label ID: 3 => Original Name:  Fearful
Label ID: 4 => Original Name:  Happy
Label ID: 5 => Original Name:  Neutral
Label ID: 6 => Original Name:  Sad
Label ID: 7 => Original Name:  Surprised


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/EDA/Ai Prompt Generator/Ai Writting/Conversation/topical_chat.csv')

In [ ]:
# Sample 2000 random rows from the DataFrame
sampled_data = data.sample(n=1000, random_state=42)
# Display the sampled data
print(sampled_data.head())

        conversation_id                                            message  \
13494               618   Yes we would.  For something odd closer to ho...   
67785              3103   I have never seen that movie, though I heard ...   
27310              1251   Hi there ! Looking forward to chat with You ! ;)   
173536             7950   He also has a supermodel for a wife. That doe...   
151263             6928   I think they are all doing pretty well. In fa...   

       sentiment  
13494    Neutral  
67785    Neutral  
27310      Happy  
173536     Happy  
151263     Happy  


In [ ]:
data = sampled_data

In [ ]:
import pandas as pd
import torch


# Select the column 'question' from the dataset
questions = data['message'].tolist()  # Convert the questions to a list

# Tokenize the questions
encoded_inputs = tokenizer(questions, padding=True, truncation=True, return_tensors="pt", max_length=512)

# Prepare the input tensors
input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]

# Set the device to use (e.g., CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the tensors to the specified device
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Predict the sentiments for the questions
with torch.no_grad():
    logits = model(input_ids, attention_mask=attention_mask)[0]
    predictions = torch.argmax(logits, dim=1).cpu().numpy()

# Add the predicted sentiments as a new column in the dataset
data['BERT_sentiment'] = predictions

# # Save the updated dataset with the new sentiment column
#


In [ ]:
data.to_csv("tmp_data_with_sentiments_Test.csv", index=False)

In [ ]:
# Define the mapping of label IDs to original names
label_mapping = {
    0: "Angry",
    1: "Curious to dive deeper",
    2: "Sad",
    3: "Fearful",
    4: "Neutral",
    5: "Happy",
    6: "Sad",
    7: "Surprised",
    8: "Disgusted"
}

# Replace the sentiment labels in the data with the original names
data['BERT_sentiment'] = data['BERT_sentiment'].map(label_mapping)

In [ ]:
new_text = [" Haha! Woah! There's a thing called transhumanists I guess, and they want to upload their brains to computers and memory sticks! I don't want to do that! One lifetime is okay by me. I can only imagine the torment of a mind having to just live for possibly forever! Yikes!",
            " this anxites me , Yeah no way. I hate how it ruined his reputation. At least he apologized." ,
            "   Haha! I am quite happy keeping my thoughts to myself! It sounds like a scary sci-fi movie to me! My computer brain gets arrested and imprisoned for eternity in 100 years for some crazy new law! No thank you!"
            , " I wouldn't make a good bird!  Couldn't do that, at all."]

# Tokenize the text
encoded_inputs = tokenizer(new_text, padding=True, truncation=True, return_tensors="pt")

# Prepare the input tensors
input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]

# Set the device to use (e.g., CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the tensors to the specified device
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

In [ ]:
# Predict the sentiments for the input text
with torch.no_grad():
    logits = model(input_ids, attention_mask=attention_mask)[0]
    predictions = torch.argmax(logits, dim=1).cpu().numpy()

# Print the predicted sentiments for each input text
for text, sentiment in zip(new_text, predictions):
    print(f"Text: {text}\nPredicted Sentiment: {sentiment}\n")


Text:  Haha! Woah! There's a thing called transhumanists I guess, and they want to upload their brains to computers and memory sticks! I don't want to do that! One lifetime is okay by me. I can only imagine the torment of a mind having to just live for possibly forever! Yikes!
Predicted Sentiment: 1

Text:  this anxites me , Yeah no way. I hate how it ruined his reputation. At least he apologized.
Predicted Sentiment: 1

Text:    Haha! I am quite happy keeping my thoughts to myself! It sounds like a scary sci-fi movie to me! My computer brain gets arrested and imprisoned for eternity in 100 years for some crazy new law! No thank you!
Predicted Sentiment: 1

Text:  I wouldn't make a good bird!  Couldn't do that, at all.
Predicted Sentiment: 5



In [ ]:
import joblib
import pandas as pd

# Load the trained model
classifier = joblib.load("/content/GBT_Sentiment_model.pkl")

# Extract the questions from the dataset
questions = data['message']

# Predict the sentiments using the pre-trained model
sentiments = classifier.predict(questions)

# Add the predicted sentiments as a new column in the dataset
data['GBT_sentiment'] = sentiments

# Save the updated dataset with the new sentiment column
data.to_csv("data_with_sentiments.csv", index=False)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Extract the predicted and ground truth labels from the DataFrame
predicted_labels = data['sentiment']
ground_truth_labels = data['GBT_sentiment']

# Calculate evaluation metrics
accuracy = accuracy_score(ground_truth_labels, predicted_labels)
precision = precision_score(ground_truth_labels, predicted_labels, average='macro')
recall = recall_score(ground_truth_labels, predicted_labels, average='macro')
f1 = f1_score(ground_truth_labels, predicted_labels, average='macro')

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Accuracy: 0.4790
Precision: 0.3262
Recall: 0.5396
F1-Score: 0.3678


#Not Satisfied with Bert , So going on with Gradient Boost Classifier